In [ ]:
! pip install torch
! pip install git+https://github.com/huggingface/transformers
! pip install git+https://github.com/huggingface/accelerate
!pip install Pillow
# !pip install pandas
# !pip install OpenCV-python
!pip install flask_cors
!pip install pyngrok



  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-yk07yl15
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-yk07yl15
  Resolved https://github.com/huggingface/transformers to commit 85eb3392318fc91a97692f23e1ce69b916567185
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 18.8 MB/s eta 0:00:00
  Created wheel for transformers: filename=transformers-4.48.0.dev0-py3-none-any.whl size=10214389 sha256=5e238d703e0f1eeecbb90fbb36105296aea2b5eb431c0910f83acbc04003b207
  Stored in directory: /tmp/pip-ephem-wheel-cache-vhh8g1ab/wheels/c0/14/d6/6c9a5582d2ac191ec0a483be151a4495fe1eb2a6706ca49f1b
Successfully built transformers
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.20.3
    Uninstalling tokenizers-0.20.3:
      Successfully 

## Load quantized model

In [ ]:

from unsloth import FastVisionModel
model, tokenizer = FastVisionModel.from_pretrained(
        model_name = "lora_model", # YOUR MODEL YOU USED FOR TRAINING
        load_in_4bit = True, # Set to False for 16bit LoRA
    )
FastVisionModel.for_inference(model) # Enable for inference!

from transformers import MllamaForConditionalGeneration, AutoProcessor, BitsAndBytesConfig
import torch
import requests
from PIL import Image


# Setup 4-bit quantization
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.bfloat16  # Using bf16 for better speed and efficiency
)

# Load model with quantization
model = MllamaForConditionalGeneration.from_pretrained(
    model,
    quantization_config=quantization_config,
    device_map="auto"
)

processor = AutoProcessor.from_pretrained(model)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/56.4k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/3.99G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/429M [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/4.19k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

chat_template.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

## Getting Data Detect, identify, and count all distinct grocery items present in the given image

In [ ]:
import torch
import concurrent.futures
from PIL import Image
# Function to prepare inputs and run inference with timeout
prompt = f"""Task: Detect, identify, and count all distinct grocery items present in the given image.
    Instructions:
    Count all items: Identify and count every visible grocery item in the image, ensuring the total count is as high as possible, even if item names or expiry dates cannot be identified.
    Detect brand names: Recognize and list the brand names of items, if possible. If multiple items of the same brand are present, provide the count for that brand.
    Detect expiry dates: For each brand, attempt to recognize the expiry date of the items as dd/mm/yyyy, if visible. If no expiry date can be identified, mark it as "NA".
    Important Rules for the Output:
    strictly DO NOT include any extra text, explanations, or comments other than the output format.
    If expiry date is "NA", then "Expired" and "Expected life span (Days)" should be "NA". 
    Output format: json
    [
      {{
        "TotalItems":{{total_no_of_items}}
      }},
      {{
        "Sl no": {{serial_number}},
        "Brand": "{{brand_name}}",
        "Expiry date": "{{expiry_date or NA}}",
        "Count": {{count}}
      }}
    ]"""
def run_wout_ocr(image, timeout=60):
    messages = [
    {
        "role": "user",
        "content": [
            {
                "type": "image",
                "image": image,  # Your image object
            },
            {
                "type": "text",
                "text": prompt
            },
        ],
    }
]


    # Prepare inputs
    text = processor.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    image_inputs, video_inputs = process_vision_info(messages)
    inputs = processor(
        text=[text],
        images=image_inputs,
        videos=video_inputs,
        padding=True,
        return_tensors="pt",
    )
    inputs = inputs.to("cuda")

    def model_inference():
        with torch.no_grad():
            generated_ids = model.generate(**inputs, max_new_tokens=128)
            generated_ids_trimmed = [
                out_ids[len(in_ids):] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
            ]
            output_text = processor.batch_decode(
                generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
            )
            return output_text[0]

    try:
        # Use concurrent futures for timeout handling
        with concurrent.futures.ThreadPoolExecutor() as executor:
            future = executor.submit(model_inference)
            output = future.result(timeout=timeout)  # timeout in seconds
            return output

    except concurrent.futures.TimeoutError:
        print(f"Inference timed out after {timeout} seconds.")
        return {"error": "Inference timed out"}
    except Exception as e:
        print(f"Inference failed: {e}")
        return {"error": f"Inference failed: {str(e)}"}


In [ ]:
import os
from PIL import Image

# Open the image
image = Image.open('/content/drive/MyDrive/###DL AND ML MODELS /Qwen2-VL-for-OCR-VQA-main-20241005T160243Z-001/1.jpg')

# Pass the image to the run_inference function to extract text
# text = ocr_paddle(image_path)
# print(text)
extracted_text = run_wout_ocr(image)

# Print the extracted text
print(f"{extracted_text}")
print("------------------------------------")


## Make A Server Through Network Tunneling

In [ ]:
from flask import Flask, request, jsonify
from pyngrok import ngrok
from flask_cors import CORS
from PIL import Image
import io

# Set your ngrok authentication token
ngrok.set_auth_token("2q7eRWihbOqNivIgBoSPCmiWAbU_4bpeBcbppUtb5wwYQeYuJ")

app = Flask(__name__)

# Enable CORS for the app
CORS(app)

# Your OCR processing function

# Common function to process image
def process_image_request(image_file):
    if not image_file:
        return {"error": "No image file provided"}, 400

    if image_file.filename == '':
        return {"error": "No image selected"}, 400

    try:
        # Read the image using PIL
        image = Image.open(image_file)

        # Process the image using the OCR function
        result = run_wout_ocr(image)
        print(result)
        return result, 200

    except Exception as e:
        return {"error": str(e)}, 500

# Endpoints
@app.route('/webhook1', methods=['POST'])
def webhook():
    image_file = request.files.get('file')
    response, status_code = process_image_request(image_file)
    print(response)
    return jsonify(response), status_code


In [ ]:
ngrok_tunnel = ngrok.connect(5000)
print("Ngrok URL:", ngrok_tunnel.public_url)

# Run the Flask app
app.run(port=5000, debug=True, use_reloader=False)

Ngrok URL: https://37c6-34-125-208-109.ngrok-free.app
 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [16/Dec/2024 12:43:44] "POST /webhook1 HTTP/1.1" 200 -


```json
[
  {
    "TotalItems": 1
  },
  {
    "Sl no": 1,
    "Brand": "Maggi",
    "Expiry date": "NA",
    "Count": 10
  }
]
```
```json
[
  {
    "TotalItems": 1
  },
  {
    "Sl no": 1,
    "Brand": "Maggi",
    "Expiry date": "NA",
    "Count": 10
  }
]
```


INFO:werkzeug:127.0.0.1 - - [16/Dec/2024 13:05:21] "POST /webhook1 HTTP/1.1" 200 -


```json
[
  {
    "TotalItems": 1
  },
  {
    "Sl no": 1,
    "Brand": "Maggi",
    "Expiry date": "NA",
    "Count": 10
  }
]
```
```json
[
  {
    "TotalItems": 1
  },
  {
    "Sl no": 1,
    "Brand": "Maggi",
    "Expiry date": "NA",
    "Count": 10
  }
]
```


INFO:werkzeug:127.0.0.1 - - [16/Dec/2024 13:09:45] "POST /webhook1 HTTP/1.1" 200 -


{
  "TotalItems": 28,
  "Sl no": 1,
  "Brand": "Tata",
  "Expiry date": "NA",
  "Count": 1
}
{
  "TotalItems": 28,
  "Sl no": 1,
  "Brand": "Tata",
  "Expiry date": "NA",
  "Count": 1
}


INFO:werkzeug:127.0.0.1 - - [16/Dec/2024 13:10:36] "POST /webhook1 HTTP/1.1" 200 -


{
  "TotalItems": 28,
  "Sl no": 1,
  "Brand": "Tata",
  "Expiry date": "NA",
  "Count": 1
}
{
  "TotalItems": 28,
  "Sl no": 1,
  "Brand": "Tata",
  "Expiry date": "NA",
  "Count": 1
}
